In [1]:
import numpy as np  # http://www.numpy.org
import matplotlib.pyplot as plt   # http://matplotlib.org
import math as M  # https://docs.python.org/2/library/math.html

In [7]:
############ Initilisation #################
############################################

#### Constants #####
L_x = 1e6                                          # [m] zonal extent 
L_y = 2e6                                          # [m] meridional extent 
A = L_x*L_y                                        # [m^2] surface area of interior 
P = 2*L_x+2*L_y                                    # [m] Perimeter 

g = 9.81                                           # [m/s^2] gravitational acceleration 
C_p = 3.994e3                                      # [J/(kg K)] specific heat capacity of sea water 
s = 0.02                                           # [] average bottom slope around the perimeter of the marginal sea
f_0 = 1.2e-4                                       # [s^-1] Coriolis parameter 
alpha_T = 0.2                                      # [kg m^-3 C^-1] thermal expansion coefficient; source: Ypma[2020]
rho_ref = 1023.6                                   # [kg m^-3] reference density sea water; source: Wikipedia
T_ref = 25                                         # [Celcius] reference temperature sea water; source: Wikipedia

#### Parameters ####
num_val = 10
A_T_1 = np.append(8.87, np.linspace(7,11,num_val))       # [Celsius] ingoing temperature 
A_T_A = np.append(2.8, np.linspace(0,7,num_val))         # [Celsius] Atmospheric temperature 
A_E = np.append(-2e-8,np.linspace(0,-5e-8,num_val))      # [ppt] net evaporation minus precipitation
A_Gamma = np.append(10,np.linspace(4,60,num_val))        # [W/(m^2 C)] Restoring strength 
A_H = np.append(1000, np.linspace(600,1200,num_val))     # [m] sill height 
A_c = np.append(0.006, np.linspace(0.003,0.009,num_val)) # [] eddy coefficient 

#### Preparation #####
T_0 = np.zeros(num_val)
T_2 = np.zeros(num_val)
A_rho_0 = np.zeros(num_val)
A_rho_1 = np.zeros(num_val)
A_rho_2 = np.zeros(num_val)
A_W = np.zeros(num_val)
Delta_T = np.zeros(num_val)

In [8]:
############# Computation ##################
############################################

#### Parameterization ####
def func_epsilon(c):
    return c*P/L

def func_mu(Gamma, H, T_1, T_A):
    return A*Gamma*f_0/(alpha_T*g*C_p*H**2*(T_1-T_A)) #[-]

#### Two main functions #####
def Temp_0(T_1, mu, epsilon, T_A):
    return T_1 - (mu/epsilon)*(M.sqrt(1+2*epsilon/mu)-1)*(T_1 - T_A)

def Temp_2(T_1, T_0, mu, T_A, L):
    return T_1 - 2*mu*((T_1-T_A)/(T_1-T_0)*(1+P*L/A)-1)*(T_1-T_A)

#### Follow up computations ####
def bc_velocity(H, T_1, T_0, L):
    return alpha_T*g*H*(T_1-T_0)/(2*rho_ref*f_0*L)

def downwelling(H, T_1, T_0, T_2):
    return alpha_T*g*H**2*(T_1-T_0)/(4*rho_ref*f_0)*(1-((T_2-T_0)/(T_1-T_0)))

def density(T):
    return rho_ref+(alpha_T*(T-T_ref))

#def Q(T_0, T_A, L, Gamma, T_1) :
#    return A*(T_0 - T_A)*Gamma + P*L*Gamma*(T_1-T_A)
#def Delta(T_0, T_1, T_2):
#    return (T_2-T_0)/(T_1-T_0)

In [14]:
################ Output ####################
############################################

y=5

if y==1:          ######## Varying Eddy Coefficient c ##########
    for i in range(num_val):
        c, T_1, T_A, E, Gamma, H  = A_c[1::][i], A_T_1[0], A_T_A[0], A_E[0], A_Gamma[0], A_H[0] 
        L = H/s  # [m] width of sloping topography of the boundary current 
        epsilon = func_epsilon(c)
        mu = func_mu(Gamma, H, T_1, T_A)
        T_0[i]=Temp_0(T_1, mu, epsilon, T_A)
        T_2[i]=Temp_2(T_1, T_0[i], mu, T_A, L)
        A_rho_0[i]=density(T_0[i])
        A_rho_2[i]=density(T_2[i])
        
    var_1_1, var_1_2, var_1_3, var_1_4 = T_0, T_2, A_rho_0, A_rho_2
    %store var_1_1
    %store var_1_2
    %store var_1_3
    %store var_1_4
    
elif y==2:
    for i in range(num_val):
        c, T_1, T_A, E, Gamma, H  = A_c[0], A_T_1[0],  A_T_A[0], A_E[0], A_Gamma[1::][i], A_H[0] 
        L = H/s  # [m] width of sloping topography of the boundary current 
        
        epsilon = func_epsilon(c)
        mu = func_mu(Gamma, H, T_1, T_A)
        T_0[i]=Temp_0(T_1, mu, epsilon, T_A)
        T_2[i]=Temp_2(T_1, T_0[i], mu, T_A, L)
        A_rho_0[i]=density(T_0[i])
        A_rho_2[i]=density(T_2[i])
    var_2_1, var_2_2, = T_0, T_2
    %store var_2_1
    %store var_2_2
    
if y==3:          ######## Delta T ##########
    for i in range(num_val):
        c, T_1, T_A, E, Gamma, H  = A_c[1::][i], A_T_1[0], A_T_A[0], A_E[0], A_Gamma[0], A_H[0] 
        L = H/s  # [m] width of sloping topography of the boundary current 
        
        epsilon = func_epsilon(c)
        mu = func_mu(Gamma, H, T_1, T_A)
        T_0[i]=Temp_0(T_1, mu, epsilon, T_A)
        Delta_T[i] = T_1-T_0[i]
    var_3 = Delta_T
    %store var_3
    
if y==4:          ######## Delta T ##########
    for i in range(num_val):
        c, T_1, T_A, E, Gamma, H  = A_c[0], A_T_1[0], A_T_A[0], A_E[0], A_Gamma[0], A_H[1::][i] 
        L = H/s  # [m] width of sloping topography of the boundary current 
        
        epsilon = func_epsilon(c)
        mu = func_mu(Gamma, H, T_1, T_A)
        T_0[i]=Temp_0(T_1, mu, epsilon, T_A)
        T_2[i]=Temp_2(T_1, T_0[i], mu, T_A, L)
        A_W[i]=downwelling(H, T_1, T_0[i], T_2[i])
    var_4 = A_W
    %store var_4

if y==5:          ######## Downwelling with incoming Tempereature ##########
    for i in range(num_val):
        c, T_1, T_A, E, Gamma, H  = A_c[0], A_T_1[1::][i], A_T_A[0], A_E[0], A_Gamma[0], A_H[0]
        L = H/s  # [m] width of sloping topography of the boundary current 
        epsilon = func_epsilon(c)
        mu = func_mu(Gamma, H, T_1, T_A)
        T_0[i]=Temp_0(T_1, mu, epsilon, T_A)
        T_2[i]=Temp_2(T_1, T_0[i], mu, T_A, L)
        A_W[i]=downwelling(H, T_1, T_0[i], T_2[i])
        
    var_5 = A_W
    %store var_5

Stored 'var_5' (ndarray)
